# Apprentissage sur les images

Entraine le modèle sur des images et pas sur les vidéos.

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline
plt.rcParams['figure.figsize'] = 12, 8
%config InlineBackend.figure_format = 'retina'

In [24]:
datagen = ImageDataGenerator(preprocessing_function=keras.applications.resnet_v2.preprocess_input)

train_dataset = datagen.flow_from_directory(
        'DB/train/',
        target_size=(224, 224),
        batch_size=32,
        classes=None,
        class_mode='categorical')

test_dataset = datagen.flow_from_directory(
        'DB/test/',
        target_size=(224, 224),
        batch_size=32,
        classes=None,
        class_mode='categorical')

Found 4736 images belonging to 5 classes.
Found 3568 images belonging to 5 classes.


## Modèle de classification
Les images sont
- peu nombreuses
- parfois floues
- une main recouvre en partie les objets
- d'autres objets sont visibles en arrière plan

In [26]:
from keras.applications import VGG16
from keras.applications import ResNet101V2
from keras.models import Sequential
from keras.layers import Input, Flatten, Dense, Dropout

resnet = ResNet101V2(include_top=False, weights="imagenet")
resnet.trainable = False

model = Sequential([
    Input((224, 224, 3)),
    resnet,
    Flatten(),
    Dense(256, activation="relu"),
    Dropout(0.1),
    Dense(256),
    Dense(5, activation="softmax")
])

modele.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet101v2 (Functional)     (None, None, None, 2048)  42626560  
_________________________________________________________________
flatten_2 (Flatten)          (None, 100352)            0         
_________________________________________________________________
dense_6 (Dense)              (None, 256)               25690368  
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_8 (Dense)              (None, 5)                 1285      
Total params: 68,384,005
Trainable params: 25,757,445
Non-trainable params: 42,626,560
_________________________________

In [27]:
from keras.optimizers import Adam
from keras.losses import categorical_crossentropy

optimiser = Adam()
model.compile(optimizer=optimiser, loss=categorical_crossentropy, metrics=["accuracy"])
history = model.fit(train_dataset)
model.save("modele-resnet")

148/148 [==============================] - 531s 4s/step - loss: 2.1869 - accuracy: 0.9415
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: modele-resnet\assets


In [28]:
best_model = keras.models.load_model("modele-resnet")

In [29]:
res = best_model.predict(test_dataset, verbose=1)

112/112 [==============================] - 374s 3s/step


In [34]:
print(f"Prediction result shape: {res.shape}")
y_classes = res.argmax(axis=-1)
print(y_classes.shape)

Prediction result shape: (3568, 5)
